In [48]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
import scipy
import xgboost as xgb

In [3]:
import nltk

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

In [4]:
car_train = pd.read_excel('D:\\OSSI\\Others\\MachineHack\\Used_Cars\\InputFiles\\Participants_Data_Used_Cars\\Data_Train.xlsx')
print(car_train)

                                                  Name    Location  Year  \
0                               Maruti Wagon R LXI CNG      Mumbai  2010   
1                     Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
2                                         Honda Jazz V     Chennai  2011   
3                                    Maruti Ertiga VDI     Chennai  2012   
4                      Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
5                      Hyundai EON LPG Era Plus Option   Hyderabad  2012   
6                               Nissan Micra Diesel XV      Jaipur  2013   
7                    Toyota Innova Crysta 2.8 GX AT 8S      Mumbai  2016   
8                  Volkswagen Vento Diesel Comfortline        Pune  2013   
9                       Tata Indica Vista Quadrajet LS     Chennai  2012   
10                                    Maruti Ciaz Zeta       Kochi  2018   
11                         Honda City 1.5 V AT Sunroof     Kolkata  2012   
12          

In [5]:
car_test = pd.read_excel('D:\\OSSI\\Others\\MachineHack\\Used_Cars\\InputFiles\\Participants_Data_Used_Cars\\Data_Test.xlsx')
print(car_test)

                                                   Name    Location  Year  \
0                               Maruti Alto K10 LXI CNG       Delhi  2014   
1                         Maruti Alto 800 2016-2019 LXI  Coimbatore  2013   
2             Toyota Innova Crysta Touring Sport 2.4 MT      Mumbai  2017   
3                                  Toyota Etios Liva GD   Hyderabad  2012   
4                                     Hyundai i20 Magna      Mumbai  2014   
5                                Mahindra XUV500 W8 2WD  Coimbatore  2016   
6                   Toyota Fortuner 4x2 AT TRD Sportivo        Pune  2015   
7                                  Hyundai EON Era Plus      Jaipur  2013   
8                                   Honda City 1.5 S MT      Mumbai  2011   
9                                Mahindra XUV500 W6 2WD  Coimbatore  2015   
10                            Audi Q5 2008-2012 2.0 TDI      Mumbai  2012   
11                              Hyundai Grand i10 Magna      Jaipur  2016   

In [6]:
car_all = car_train.append(car_test).reset_index(drop=True)
#food_all.fillna(value='',inplace=True)
print(car_all)

       Engine Fuel_Type  Kilometers_Driven    Location     Mileage  \
0      998 CC       CNG              72000      Mumbai  26.6 km/kg   
1     1582 CC    Diesel              41000        Pune  19.67 kmpl   
2     1199 CC    Petrol              46000     Chennai   18.2 kmpl   
3     1248 CC    Diesel              87000     Chennai  20.77 kmpl   
4     1968 CC    Diesel              40670  Coimbatore   15.2 kmpl   
5      814 CC       LPG              75000   Hyderabad  21.1 km/kg   
6     1461 CC    Diesel              86999      Jaipur  23.08 kmpl   
7     2755 CC    Diesel              36000      Mumbai  11.36 kmpl   
8     1598 CC    Diesel              64430        Pune  20.54 kmpl   
9     1248 CC    Diesel              65932     Chennai   22.3 kmpl   
10    1462 CC    Petrol              25692       Kochi  21.56 kmpl   
11    1497 CC    Petrol              60000     Kolkata   16.8 kmpl   
12    1248 CC    Diesel              64424      Jaipur   25.2 kmpl   
13    2179 CC    Die

D:\Anaconda\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
location = car_all['Location'].value_counts()
#locality.to_csv('Locality.csv', index=True)
print(location)

Mumbai        949
Hyderabad     876
Kochi         772
Coimbatore    772
Pune          765
Delhi         660
Kolkata       654
Chennai       591
Jaipur        499
Bangalore     440
Ahmedabad     275
Name: Location, dtype: int64


In [8]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

In [9]:
count_vect = CountVectorizer(analyzer=clean_text)
X_counts = count_vect.fit_transform(car_all['Name'])
print(X_counts.shape)
print(count_vect.get_feature_names())

(7253, 894)
['1', '10', '100', '1000', '105', '107', '11', '110', '110ps', '118d', '11l', '12', '124', '12l', '12p', '12t', '13', '13l', '14', '14gsi', '14l', '15', '15d', '15p', '16', '16l', '177', '18', '18i', '19', '199', '2', '20', '200', '20012003', '20012011', '20032012', '2005', '20052011', '20062010', '20062013', '2007', '20072010', '20072011', '20072012', '20072015', '20082012', '20082013', '20092012', '20092013', '20092014', '20092016', '2010', '20102013', '20102014', '20112014', '20112015', '2012', '20122014', '20122015', '2013', '20132015', '20132016', '20132017', '2014', '20142019', '20152017', '20152019', '20162019', '2020', '20d', '20l', '210', '22', '220', '220cdibe', '220d', '22l', '23', '230', '230jm', '24', '24l', '25', '250', '250d', '250hp', '25i', '25l', '25si', '26', '27', '28', '280', '2wd', '3', '30', '300', '30d', '30l', '318i', '32', '320', '320d', '320i', '325i', '328i', '330', '330d', '330i', '35', '350', '350d', '35i', '36', '370z', '4', '40', '400', '40d'

In [10]:
X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = count_vect.get_feature_names()
X_counts_df

,1,10,100,1000,105,107,11,110,110ps,118d,...,z4,zdi,zen,zest,zeta,zipdrive,zipplus,zlx,zx,zxi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
car_all_ex = pd.concat([car_all,X_counts_df], axis=1, sort=False)
#food_all_ex = food_all.join(X_counts_df)
print(car_all_ex)

       Engine Fuel_Type  Kilometers_Driven    Location     Mileage  \
0      998 CC       CNG              72000      Mumbai  26.6 km/kg   
1     1582 CC    Diesel              41000        Pune  19.67 kmpl   
2     1199 CC    Petrol              46000     Chennai   18.2 kmpl   
3     1248 CC    Diesel              87000     Chennai  20.77 kmpl   
4     1968 CC    Diesel              40670  Coimbatore   15.2 kmpl   
5      814 CC       LPG              75000   Hyderabad  21.1 km/kg   
6     1461 CC    Diesel              86999      Jaipur  23.08 kmpl   
7     2755 CC    Diesel              36000      Mumbai  11.36 kmpl   
8     1598 CC    Diesel              64430        Pune  20.54 kmpl   
9     1248 CC    Diesel              65932     Chennai   22.3 kmpl   
10    1462 CC    Petrol              25692       Kochi  21.56 kmpl   
11    1497 CC    Petrol              60000     Kolkata   16.8 kmpl   
12    1248 CC    Diesel              64424      Jaipur   25.2 kmpl   
13    2179 CC    Die

In [12]:
print(car_all_ex.isnull().sum(axis = 0))

Engine                 46
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 2
Name                    0
New_Price            6247
Owner_Type              0
Power                  46
Price                1234
Seats                  53
Transmission            0
Year                    0
1                       0
10                      0
100                     0
1000                    0
105                     0
107                     0
11                      0
110                     0
110ps                   0
118d                    0
11l                     0
12                      0
124                     0
12l                     0
12p                     0
12t                     0
13                      0
                     ... 
xj                      0
xl                      0
xline                   0
xlt                     0
xm                      0
xms                     0
xo                      0
xp          

In [13]:
le = preprocessing.LabelEncoder()
car_all_ex['Location'] = le.fit_transform(car_all_ex['Location'])
car_all_ex['Kilometers_Driven'] = le.fit_transform(car_all_ex['Kilometers_Driven'])
car_all_ex['Fuel_Type'] = le.fit_transform(car_all_ex['Fuel_Type'])
car_all_ex['Transmission'] = le.fit_transform(car_all_ex['Transmission'])
car_all_ex['Owner_Type'] = le.fit_transform(car_all_ex['Owner_Type'])
print(car_all_ex)

       Engine  Fuel_Type  Kilometers_Driven  Location     Mileage  \
0      998 CC          0               2794         9  26.6 km/kg   
1     1582 CC          1               1344        10  19.67 kmpl   
2     1199 CC          4               1614         2   18.2 kmpl   
3     1248 CC          1               3190         2  20.77 kmpl   
4     1968 CC          1               1335         3   15.2 kmpl   
5      814 CC          3               2884         5  21.1 km/kg   
6     1461 CC          1               3189         6  23.08 kmpl   
7     2755 CC          1               1097         9  11.36 kmpl   
8     1598 CC          1               2484        10  20.54 kmpl   
9     1248 CC          1               2565         2   22.3 kmpl   
10    1462 CC          4                633         7  21.56 kmpl   
11    1497 CC          4               2263         8   16.8 kmpl   
12    1248 CC          1               2483         6   25.2 kmpl   
13    2179 CC          1          

In [14]:
car_all_ex['Mileage'] = car_all_ex['Mileage'].str.replace(' km/kg', '')
car_all_ex['Mileage'] = car_all_ex['Mileage'].str.replace(' kmpl', '')
car_all_ex['Power'] = car_all_ex['Power'].str.replace(' bhp', '')
#car_all_ex['Power'] = car_all_ex['Power'].replace('null',np.nan)
car_all_ex['Engine'] = car_all_ex['Engine'].str.replace(' CC', '')
car_all_ex

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Name,New_Price,Owner_Type,Power,Price,...,z4,zdi,zen,zest,zeta,zipdrive,zipplus,zlx,zx,zxi
0,998,0,2794,9,26.6,Maruti Wagon R LXI CNG,NaN,0,58.16,1.75,...,0,0,0,0,0,0,0,0,0,0
1,1582,1,1344,10,19.67,Hyundai Creta 1.6 CRDi SX Option,NaN,0,126.2,12.50,...,0,0,0,0,0,0,0,0,0,0
2,1199,4,1614,2,18.2,Honda Jazz V,8.61 Lakh,0,88.7,4.50,...,0,0,0,0,0,0,0,0,0,0
3,1248,1,3190,2,20.77,Maruti Ertiga VDI,NaN,0,88.76,6.00,...,0,0,0,0,0,0,0,0,0,0
4,1968,1,1335,3,15.2,Audi A4 New 2.0 TDI Multitronic,NaN,2,140.8,17.74,...,0,0,0,0,0,0,0,0,0,0
5,814,3,2884,5,21.1,Hyundai EON LPG Era Plus Option,NaN,0,55.2,2.35,...,0,0,0,0,0,0,0,0,0,0
6,1461,1,3189,6,23.08,Nissan Micra Diesel XV,NaN,0,63.1,3.50,...,0,0,0,0,0,0,0,0,0,0
7,2755,1,1097,9,11.36,Toyota Innova Crysta 2.8 GX AT 8S,21 Lakh,0,171.5,17.50,...,0,0,0,0,0,0,0,0,0,0
8,1598,1,2484,10,20.54,Volkswagen Vento Diesel Comfortline,NaN,0,103.6,5.20,...,0,0,0,0,0,0,0,0,0,0
9,1248,1,2565,2,22.3,Tata Indica Vista Quadrajet LS,NaN,2,74,1.95,...,0,0,0,0,0,0,0,0,0,0


In [15]:
car_all_ex["Location"] = pd.to_numeric(car_all_ex["Location"])
car_all_ex["Year"] = pd.to_numeric(car_all_ex["Year"])
car_all_ex["Kilometers_Driven"] = pd.to_numeric(car_all_ex["Kilometers_Driven"])
car_all_ex["Fuel_Type"] = pd.to_numeric(car_all_ex["Fuel_Type"])
car_all_ex["Transmission"] = pd.to_numeric(car_all_ex["Transmission"])
car_all_ex["Owner_Type"] = pd.to_numeric(car_all_ex["Owner_Type"])
car_all_ex["Mileage"] = pd.to_numeric(car_all_ex["Mileage"])
car_all_ex["Engine"] = pd.to_numeric(car_all_ex["Engine"])

car_all_ex["Seats"] = pd.to_numeric(car_all_ex["Seats"])

In [16]:
car_all_ex = car_all_ex.drop(['New_Price'],axis=1)
car_all_ex

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Name,Owner_Type,Power,Price,Seats,...,z4,zdi,zen,zest,zeta,zipdrive,zipplus,zlx,zx,zxi
0,998.0,0,2794,9,26.60,Maruti Wagon R LXI CNG,0,58.16,1.75,5.0,...,0,0,0,0,0,0,0,0,0,0
1,1582.0,1,1344,10,19.67,Hyundai Creta 1.6 CRDi SX Option,0,126.2,12.50,5.0,...,0,0,0,0,0,0,0,0,0,0
2,1199.0,4,1614,2,18.20,Honda Jazz V,0,88.7,4.50,5.0,...,0,0,0,0,0,0,0,0,0,0
3,1248.0,1,3190,2,20.77,Maruti Ertiga VDI,0,88.76,6.00,7.0,...,0,0,0,0,0,0,0,0,0,0
4,1968.0,1,1335,3,15.20,Audi A4 New 2.0 TDI Multitronic,2,140.8,17.74,5.0,...,0,0,0,0,0,0,0,0,0,0
5,814.0,3,2884,5,21.10,Hyundai EON LPG Era Plus Option,0,55.2,2.35,5.0,...,0,0,0,0,0,0,0,0,0,0
6,1461.0,1,3189,6,23.08,Nissan Micra Diesel XV,0,63.1,3.50,5.0,...,0,0,0,0,0,0,0,0,0,0
7,2755.0,1,1097,9,11.36,Toyota Innova Crysta 2.8 GX AT 8S,0,171.5,17.50,8.0,...,0,0,0,0,0,0,0,0,0,0
8,1598.0,1,2484,10,20.54,Volkswagen Vento Diesel Comfortline,0,103.6,5.20,5.0,...,0,0,0,0,0,0,0,0,0,0
9,1248.0,1,2565,2,22.30,Tata Indica Vista Quadrajet LS,2,74,1.95,5.0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
type(car_all_ex.loc[:,'Power'])

pandas.core.series.Series

In [18]:
nndf = car_all_ex.loc[car_all_ex['Engine'].notnull() & car_all_ex['Power'].notnull(),['Engine','Power']]
nndf

,Engine,Power
0,998.0,58.16
1,1582.0,126.2
2,1199.0,88.7
3,1248.0,88.76
4,1968.0,140.8
5,814.0,55.2
6,1461.0,63.1
7,2755.0,171.5
8,1598.0,103.6
9,1248.0,74


In [19]:
nndf= nndf.loc[nndf['Power']!= 'null']
nndf

,Engine,Power
0,998.0,58.16
1,1582.0,126.2
2,1199.0,88.7
3,1248.0,88.76
4,1968.0,140.8
5,814.0,55.2
6,1461.0,63.1
7,2755.0,171.5
8,1598.0,103.6
9,1248.0,74


In [20]:
nndf["Power"] = pd.to_numeric(nndf["Power"])
nndf['ratio'] = nndf['Engine']/nndf['Power']
ratmean = nndf['ratio'].mean()
ratmean

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


15.148696665288368

In [21]:
car_all_ex

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Name,Owner_Type,Power,Price,Seats,...,z4,zdi,zen,zest,zeta,zipdrive,zipplus,zlx,zx,zxi
0,998.0,0,2794,9,26.60,Maruti Wagon R LXI CNG,0,58.16,1.75,5.0,...,0,0,0,0,0,0,0,0,0,0
1,1582.0,1,1344,10,19.67,Hyundai Creta 1.6 CRDi SX Option,0,126.2,12.50,5.0,...,0,0,0,0,0,0,0,0,0,0
2,1199.0,4,1614,2,18.20,Honda Jazz V,0,88.7,4.50,5.0,...,0,0,0,0,0,0,0,0,0,0
3,1248.0,1,3190,2,20.77,Maruti Ertiga VDI,0,88.76,6.00,7.0,...,0,0,0,0,0,0,0,0,0,0
4,1968.0,1,1335,3,15.20,Audi A4 New 2.0 TDI Multitronic,2,140.8,17.74,5.0,...,0,0,0,0,0,0,0,0,0,0
5,814.0,3,2884,5,21.10,Hyundai EON LPG Era Plus Option,0,55.2,2.35,5.0,...,0,0,0,0,0,0,0,0,0,0
6,1461.0,1,3189,6,23.08,Nissan Micra Diesel XV,0,63.1,3.50,5.0,...,0,0,0,0,0,0,0,0,0,0
7,2755.0,1,1097,9,11.36,Toyota Innova Crysta 2.8 GX AT 8S,0,171.5,17.50,8.0,...,0,0,0,0,0,0,0,0,0,0
8,1598.0,1,2484,10,20.54,Volkswagen Vento Diesel Comfortline,0,103.6,5.20,5.0,...,0,0,0,0,0,0,0,0,0,0
9,1248.0,1,2565,2,22.30,Tata Indica Vista Quadrajet LS,2,74,1.95,5.0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
def get_power(df):
    #if df['CITY'] in ('Bangalore','Bangalore.'):
    if df['Power'] == 'null':
        return df['Engine']/ratmean
    else:
        return df['Power']

car_all_ex['Power'] = car_all_ex.apply(get_power, axis = 1)


In [23]:
car_all_ex.iloc[76]

Engine                                   1399
Fuel_Type                                   1
Kilometers_Driven                        3428
Location                                    6
Mileage                                  17.8
Name                 Ford Fiesta 1.4 SXi TDCi
Owner_Type                                  0
Power                                 92.3512
Price                                       2
Seats                                       5
Transmission                                1
Year                                     2008
1                                           0
10                                          0
100                                         0
1000                                        0
105                                         0
107                                         0
11                                          0
110                                         0
110ps                                       0
118d                              

In [24]:
print(car_all_ex.isnull().sum(axis = 0))

Engine                 46
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 2
Name                    0
Owner_Type              0
Power                  46
Price                1234
Seats                  53
Transmission            0
Year                    0
1                       0
10                      0
100                     0
1000                    0
105                     0
107                     0
11                      0
110                     0
110ps                   0
118d                    0
11l                     0
12                      0
124                     0
12l                     0
12p                     0
12t                     0
13                      0
13l                     0
                     ... 
xj                      0
xl                      0
xline                   0
xlt                     0
xm                      0
xms                     0
xo                      0
xp          

In [25]:
car_all_ex["Power"] = pd.to_numeric(car_all_ex["Power"])

eng_mean = car_all_ex.loc[car_all_ex['Engine'].notnull(),'Engine'].mean()
mil_mean = car_all_ex.loc[car_all_ex['Mileage'].notnull() | car_all_ex['Mileage'] != 0,'Mileage'].mean()
power_mean = car_all_ex.loc[car_all_ex['Power'].notnull(),'Power'].mean()
seat_mean = 5

type(eng_mean)

float

In [26]:
def zero_mileage(df):
    if df['Mileage'] == 0:
        return mil_mean
    else:
        return df['Mileage']

car_all_ex['Mileage'] = car_all_ex.apply(zero_mileage, axis = 1)

print(car_all_ex.isnull().sum(axis = 0))

Engine                 46
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 2
Name                    0
Owner_Type              0
Power                  46
Price                1234
Seats                  53
Transmission            0
Year                    0
1                       0
10                      0
100                     0
1000                    0
105                     0
107                     0
11                      0
110                     0
110ps                   0
118d                    0
11l                     0
12                      0
124                     0
12l                     0
12p                     0
12t                     0
13                      0
13l                     0
                     ... 
xj                      0
xl                      0
xline                   0
xlt                     0
xm                      0
xms                     0
xo                      0
xp          

In [27]:
def get_null_eng_mean(df):
    if pd.isna(df['Engine']):
        return eng_mean
    else:
        return df['Engine']

def get_null_power_mean(df):
    if pd.isna(df['Power']):
        return power_mean
    else:
        return df['Power']
    
def get_null_mileage_mean(df):
    if pd.isna(df['Mileage']):
        return mil_mean
    else:
        return df['Mileage']
    
def get_null_seat_mean(df):
    if pd.isna(df['Seats']):
        return seat_mean
    else:
        return df['Seats']

car_all_ex['Engine'] = car_all_ex.apply(get_null_eng_mean,axis=1)
car_all_ex['Power'] = car_all_ex.apply(get_null_power_mean,axis=1)
car_all_ex['Mileage'] = car_all_ex.apply(get_null_mileage_mean,axis=1)
car_all_ex['Seats'] = car_all_ex.apply(get_null_seat_mean,axis=1)

In [28]:
car_all_ex = car_all_ex.drop(['Name'],axis=1)
car_all_ex 

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Owner_Type,Power,Price,Seats,Transmission,...,z4,zdi,zen,zest,zeta,zipdrive,zipplus,zlx,zx,zxi
0,998.0,0,2794,9,26.60000,0,58.16,1.75,5.0,1,...,0,0,0,0,0,0,0,0,0,0
1,1582.0,1,1344,10,19.67000,0,126.20,12.50,5.0,1,...,0,0,0,0,0,0,0,0,0,0
2,1199.0,4,1614,2,18.20000,0,88.70,4.50,5.0,1,...,0,0,0,0,0,0,0,0,0,0
3,1248.0,1,3190,2,20.77000,0,88.76,6.00,7.0,1,...,0,0,0,0,0,0,0,0,0,0
4,1968.0,1,1335,3,15.20000,2,140.80,17.74,5.0,0,...,0,0,0,0,0,0,0,0,0,0
5,814.0,3,2884,5,21.10000,0,55.20,2.35,5.0,1,...,0,0,0,0,0,0,0,0,0,0
6,1461.0,1,3189,6,23.08000,0,63.10,3.50,5.0,1,...,0,0,0,0,0,0,0,0,0,0
7,2755.0,1,1097,9,11.36000,0,171.50,17.50,8.0,0,...,0,0,0,0,0,0,0,0,0,0
8,1598.0,1,2484,10,20.54000,0,103.60,5.20,5.0,1,...,0,0,0,0,0,0,0,0,0,0
9,1248.0,1,2565,2,22.30000,2,74.00,1.95,5.0,1,...,0,0,0,0,0,0,0,0,0,0


In [29]:
print(car_all_ex.isnull().sum(axis = 0))

Engine                  0
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 0
Owner_Type              0
Power                   0
Price                1234
Seats                   0
Transmission            0
Year                    0
1                       0
10                      0
100                     0
1000                    0
105                     0
107                     0
11                      0
110                     0
110ps                   0
118d                    0
11l                     0
12                      0
124                     0
12l                     0
12p                     0
12t                     0
13                      0
13l                     0
14                      0
                     ... 
xj                      0
xl                      0
xline                   0
xlt                     0
xm                      0
xms                     0
xo                      0
xp          

In [30]:
car_all_ex

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Owner_Type,Power,Price,Seats,Transmission,...,z4,zdi,zen,zest,zeta,zipdrive,zipplus,zlx,zx,zxi
0,998.0,0,2794,9,26.60000,0,58.16,1.75,5.0,1,...,0,0,0,0,0,0,0,0,0,0
1,1582.0,1,1344,10,19.67000,0,126.20,12.50,5.0,1,...,0,0,0,0,0,0,0,0,0,0
2,1199.0,4,1614,2,18.20000,0,88.70,4.50,5.0,1,...,0,0,0,0,0,0,0,0,0,0
3,1248.0,1,3190,2,20.77000,0,88.76,6.00,7.0,1,...,0,0,0,0,0,0,0,0,0,0
4,1968.0,1,1335,3,15.20000,2,140.80,17.74,5.0,0,...,0,0,0,0,0,0,0,0,0,0
5,814.0,3,2884,5,21.10000,0,55.20,2.35,5.0,1,...,0,0,0,0,0,0,0,0,0,0
6,1461.0,1,3189,6,23.08000,0,63.10,3.50,5.0,1,...,0,0,0,0,0,0,0,0,0,0
7,2755.0,1,1097,9,11.36000,0,171.50,17.50,8.0,0,...,0,0,0,0,0,0,0,0,0,0
8,1598.0,1,2484,10,20.54000,0,103.60,5.20,5.0,1,...,0,0,0,0,0,0,0,0,0,0
9,1248.0,1,2565,2,22.30000,2,74.00,1.95,5.0,1,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# car_all_ex.to_csv('Car_final.csv', index=False)

In [32]:
car_final = car_all_ex
car_final

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Owner_Type,Power,Price,Seats,Transmission,...,z4,zdi,zen,zest,zeta,zipdrive,zipplus,zlx,zx,zxi
0,998.0,0,2794,9,26.60000,0,58.16,1.75,5.0,1,...,0,0,0,0,0,0,0,0,0,0
1,1582.0,1,1344,10,19.67000,0,126.20,12.50,5.0,1,...,0,0,0,0,0,0,0,0,0,0
2,1199.0,4,1614,2,18.20000,0,88.70,4.50,5.0,1,...,0,0,0,0,0,0,0,0,0,0
3,1248.0,1,3190,2,20.77000,0,88.76,6.00,7.0,1,...,0,0,0,0,0,0,0,0,0,0
4,1968.0,1,1335,3,15.20000,2,140.80,17.74,5.0,0,...,0,0,0,0,0,0,0,0,0,0
5,814.0,3,2884,5,21.10000,0,55.20,2.35,5.0,1,...,0,0,0,0,0,0,0,0,0,0
6,1461.0,1,3189,6,23.08000,0,63.10,3.50,5.0,1,...,0,0,0,0,0,0,0,0,0,0
7,2755.0,1,1097,9,11.36000,0,171.50,17.50,8.0,0,...,0,0,0,0,0,0,0,0,0,0
8,1598.0,1,2484,10,20.54000,0,103.60,5.20,5.0,1,...,0,0,0,0,0,0,0,0,0,0
9,1248.0,1,2565,2,22.30000,2,74.00,1.95,5.0,1,...,0,0,0,0,0,0,0,0,0,0


# Regression

In [33]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.pipeline import Pipeline

from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_log_error

from time import time

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sb

In [34]:
corr = car_final.corr()
print(corr[corr>0.2])

                     Engine  Fuel_Type  Kilometers_Driven  Location   Mileage  \
Engine             1.000000        NaN                NaN       NaN       NaN   
Fuel_Type               NaN   1.000000                NaN       NaN       NaN   
Kilometers_Driven       NaN        NaN           1.000000       NaN       NaN   
Location                NaN        NaN                NaN       1.0       NaN   
Mileage                 NaN        NaN                NaN       NaN  1.000000   
Owner_Type              NaN        NaN           0.242844       NaN       NaN   
Power              0.861096        NaN                NaN       NaN       NaN   
Price              0.657129        NaN                NaN       NaN       NaN   
Seats              0.398940        NaN                NaN       NaN       NaN   
Transmission            NaN        NaN                NaN       NaN  0.354719   
Year                    NaN        NaN                NaN       NaN  0.295609   
1                       NaN 

In [35]:
X = car_final.drop(['Price'], axis=1)

y = car_final['Price']

#X_train = X.iloc[:6019]
#X_train
#X_test = X.iloc[12690:]
#y_train = y.iloc[:6019]
#y_test = y.iloc[12690:]
#y_train
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:6019], y.iloc[:6019], test_size=.1, random_state=17)

In [36]:
from time import time

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor

regressors = [
    LinearRegression(), 
    RANSACRegressor(), 
    KNeighborsRegressor(),
    KNeighborsRegressor(n_neighbors=9, metric='manhattan'),
    GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, loss='ls'),
    SVR(),
    LinearSVR(),
    GaussianProcessRegressor(),
    SVR(kernel='linear'), # Cf. LinearSVR: much slower, might be better or worse: 
]

In [37]:
def rmsle(y, y0):
    return np.sqrt(np.mean(np.square(np.log1p(y) - np.log1p(y0))))

In [38]:
head = 6
for model in regressors[:head]:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    predictions = model.predict(X_test)
    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs)" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print("\tMean rmsl error:", rmsle(y_test,predictions))
    print()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
	Training time: 0.705s)
	Prediction time: 0.006s
	Explained variance: -256733530638.0829
	Mean absolute error: 576453.3683923113
	R2 score: -258265950005.97235
	Mean rmsl error: 1.7790314312520175



D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


RANSACRegressor(base_estimator=None, is_data_valid=None, is_model_valid=None,
        loss='absolute_loss', max_skips=inf, max_trials=100,
        min_samples=None, random_state=None, residual_threshold=None,
        stop_n_inliers=inf, stop_probability=0.99, stop_score=inf)
	Training time: 26.850s)
	Prediction time: 0.005s
	Explained variance: -2074321571938.077
	Mean absolute error: 3161411.541344322
	R2 score: -2084217814817.0693
	Mean rmsl error: 2.7315460662816924



D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')
	Training time: 0.292s)
	Prediction time: 0.246s
	Explained variance: 0.6764893518945517
	Mean absolute error: 3.3707973421926907
	R2 score: 0.6764891667877848
	Mean rmsl error: 0.3943027585363973

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='manhattan',
          metric_params=None, n_jobs=None, n_neighbors=9, p=2,
          weights='uniform')
	Training time: 0.274s)
	Prediction time: 0.217s
	Explained variance: 0.6866757916446758
	Mean absolute error: 3.3002344038390548
	R2 score: 0.6866673649365354
	Mean rmsl error: 0.38292851802085687

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
       

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
	Training time: 42.309s)
	Prediction time: 4.339s
	Explained variance: 0.1010045375050651
	Mean absolute error: 5.27024227226291
	R2 score: 0.030910399560386215
	Mean rmsl error: 0.6220150863681668



In [39]:
from sklearn.linear_model import Lasso, Ridge

lr = LinearRegression()
lasso = Lasso()
ridge = Ridge()

for model in [lr, lasso, ridge]:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(model)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print("\tMean rmsl error:", rmsle(y_test,predictions))
    print()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
	Explained variance: -256733530638.0829
	Mean absolute error: 576453.3683923113
	R2 score: -258265950005.97235
	Mean rmsl error: 1.7790314312520175



D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
	Explained variance: 0.6223879677237469
	Mean absolute error: 3.900945311653933
	R2 score: 0.6223741770644795
	Mean rmsl error: 0.5326602144300233

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


	Explained variance: 0.7707663135913088
	Mean absolute error: 2.3041725618590556
	R2 score: 0.7703061738233021
	Mean rmsl error: 0.44024079713911773



In [40]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

regressors = [LinearRegression(),
              DecisionTreeRegressor(max_depth=5),
              DecisionTreeRegressor(max_depth=10),
              DecisionTreeRegressor(max_depth=20),
              RandomForestRegressor(max_depth=30),
              RandomForestRegressor(max_depth=20),
              SVR(),]

for model in regressors:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(model)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print("\tMean rmsl error:", rmsle(y_test,predictions))
    print()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
	Explained variance: -256733530638.0829
	Mean absolute error: 576453.3683923113
	R2 score: -258265950005.97235
	Mean rmsl error: 1.7790314312520175

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
	Explained variance: 0.7482375230464746
	Mean absolute error: 2.6469832838622387
	R2 score: 0.7479857699411023
	Mean rmsl error: 0.3125937372391928



D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  


DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
	Explained variance: 0.8096778173675707
	Mean absolute error: 1.8796641073768794
	R2 score: 0.808528378637221
	Mean rmsl error: 0.22262211304341487

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
	Explained variance: 0.7743432262212459
	Mean absolute error: 1.9433630776962405
	R2 score: 0.7726166339792289
	Mean rmsl error: 0.2245689148235612



D:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
	Explained variance: 0.8176485878371019
	Mean absolute error: 1.6226315188687281
	R2 score: 0.8168674881855247
	Mean rmsl error: 0.18023980942428564



D:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
	Explained variance: 0.8008250490991732
	Mean absolute error: 1.5787861076055507
	R2 score: 0.7998307460433599
	Mean rmsl error: 0.1758644972834



D:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
	Explained variance: 0.1010045375050651
	Mean absolute error: 5.27024227226291
	R2 score: 0.030910399560386215
	Mean rmsl error: 0.6220150863681668



In [ ]:
#data_dmatrix = xgb.DMatrix(data=X,label=y)

In [91]:
#rfr = RandomForestRegressor(max_depth=30,random_state=1)
rfr = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.4, learning_rate = 0.1,
                max_depth = 40, alpha = 10, n_estimators = 160)

In [92]:
from sklearn.metrics import mean_squared_log_error, make_scorer
scorer = make_scorer(mean_squared_log_error)

from sklearn.model_selection import KFold
kf = KFold(5, random_state=0)

In [93]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(rfr, X.iloc[:6019], y.iloc[:6019], scoring=scorer, cv=kf)
print(" CV scores:", cv_scores)
print("Mean score:", np.mean(cv_scores))

D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


 CV scores: [0.03182967 0.03137857 0.03544541 0.03307807 0.03109195]
Mean score: 0.03256473260211534


In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("Xgboost", xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.4, learning_rate = 0.1,
                max_depth = 40, alpha = 10, n_estimators = 160)),
])

In [ ]:
cv_scores = cross_val_score(pipe, 
                            X.iloc[:6019], y.iloc[:6019], 
                            scoring=make_scorer(mean_squared_log_error), 
                            cv=KFold(5))

print(" Pipeline CV scores:", cv_scores)
print("Pipeline mean score:", np.mean(cv_scores))

D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [ ]:
pipe.fit(X.iloc[:6019], y.iloc[:6019])
y_pred_orig = pipe.predict(X.iloc[6019:])
y_pred_final =pd.DataFrame(y_pred_orig,columns=["Price"])
y_pred_final.to_csv('Estimated_Price.csv', index=False)